# Path Planning

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/AIResearchLab/foundations-of-robotics-labs/blob/master/2-navigation/03-path-planning.ipynb)
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/AIResearchLab/foundations-of-robotics-labs/master?filepath=2-navigation/03-path-planning.ipynb)

The third problem in navigation is to work out how to get to where you want to go.

Let's say that you now have a map and you know where you are on it. You also know where you want to go. The next step is to work out how to get there. In robotics, this problem is known as path planning. It is a very well studied problem and there are many algorithms that can be used to solve it. In this notebook, we will look at one of the simplest algorithms, the A* algorithm.

[![I am here](http://wiki.ros.org/navigation?action=AttachFile&do=get&target=nav_comic.png)](http://wiki.ros.org/navigation)

## A* Algorithm (A Star)

The A* algorithm is a very simple algorithm that can be used to find a path from a start point to a goal point. It is a very popular algorithm because it is very fast and it is guaranteed to find the shortest path if one exists. The algorithm works by searching the map for the goal point. It starts at the start point and then looks at all the points around it. It then looks at all the points around those points and so on. It keeps track of the distance from the start point to each point and the distance from each point to the goal point. It then chooses the point that is closest to the goal point and repeats the process. It keeps doing this until it finds the goal point. It then works backwards from the goal point to the start point to find the shortest path.

#### An A* algorithm example

Let's look at an example of the A* algorithm.

The source code `gist` is by *Nicholas-Swift* available [here](https://gist.github.com/Nicholas-Swift/003e1932ef2804bebef2710527008f44)

In [ ]:
"""
A* example

A small grid is created with a start and end point. The A* algorithm is used to
find the shortest path between the two points. The path is then drawn on the
grid.

Original code from: https://gist.github.com/Nicholas-Swift/003e1932ef2804bebef2710527008f44
"""


class Node():
    """A node class for A* Pathfinding"""

    def __init__(self, parent=None, position=None):
        self.parent = parent
        self.position = position

        self.g = 0
        self.h = 0
        self.f = 0

    def __eq__(self, other):
        return self.position == other.position


def astar(maze, start, end):
    """Returns a list of tuples as a path from the given start to the given end in the given maze"""

    # Create start and end node
    start_node = Node(None, start)
    start_node.g = start_node.h = start_node.f = 0
    end_node = Node(None, end)
    end_node.g = end_node.h = end_node.f = 0

    # Initialize both open and closed list
    open_list = []
    closed_list = []

    # Add the start node
    open_list.append(start_node)

    # Loop until you find the end
    while len(open_list) > 0:

        # Get the current node
        current_node = open_list[0]
        current_index = 0
        for index, item in enumerate(open_list):
            if item.f < current_node.f:
                current_node = item
                current_index = index

        # Pop current off open list, add to closed list
        open_list.pop(current_index)
        closed_list.append(current_node)

        # Found the goal
        if current_node == end_node:
            path = []
            current = current_node
            while current is not None:
                path.append(current.position)
                current = current.parent
            return path[::-1] # Return reversed path

        # Generate children
        children = []
        for new_position in [(0, -1), (0, 1), (-1, 0), (1, 0), (-1, -1), (-1, 1), (1, -1), (1, 1)]: # Adjacent squares

            # Get node position
            node_position = (current_node.position[0] + new_position[0], current_node.position[1] + new_position[1])

            # Make sure within range
            if node_position[0] > (len(maze) - 1) or node_position[0] < 0 or node_position[1] > (len(maze[len(maze)-1]) -1) or node_position[1] < 0:
                continue

            # Make sure walkable terrain
            if maze[node_position[0]][node_position[1]] != 0:
                continue

            # Create new node
            new_node = Node(current_node, node_position)

            # Append
            children.append(new_node)

        # Loop through children
        for child in children:

            # Child is on the closed list
            for closed_child in closed_list:
                if child == closed_child:
                    continue

            # Create the f, g, and h values
            child.g = current_node.g + 1
            child.h = ((child.position[0] - end_node.position[0]) ** 2) + ((child.position[1] - end_node.position[1]) ** 2)
            child.f = child.g + child.h

            # Child is already in the open list
            for open_node in open_list:
                if child == open_node and child.g > open_node.g:
                    continue

            # Add the child to the open list
            open_list.append(child)


def main():
    import matplotlib.pyplot as plt

    maze = [[0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

    start = (0, 0)
    end = (7, 6)

    path = astar(maze, start, end)

    # plot the map
    plt.figure(figsize=(10,10))
    plt.imshow(maze, cmap='gray')
    # plot the path
    for i in range(len(path)):
        plt.plot(path[i][1], path[i][0], 'r.')
    plt.show()


In [ ]:
# run the example
main()